# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning', 'R. E. Hviding']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
J. Li  ->  J. Li  |  ['J. Li']
D. Semenov  ->  D. Semenov  |  ['D. Semenov']
D. Serrano  ->  D. Serrano  |  ['D. Serrano']


Arxiv has 58 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2602.06116
extracting tarball to tmp_2602.06116...

 done.


Retrieving document from  https://arxiv.org/e-print/2602.06463


extracting tarball to tmp_2602.06463... done.
Retrieving document from  https://arxiv.org/e-print/2602.06551


extracting tarball to tmp_2602.06551... done.


D. Semenov  ->  D. Semenov  |  ['D. Semenov']


Retrieving document from  https://arxiv.org/e-print/2602.06878
extracting tarball to tmp_2602.06878... done.


/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2602.06878/Coil_paper_iop.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'include/Appendix' from 'tmp_2602.06878/include/Appendix.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'include/Acknowledgements' from 'tmp_2602.06878/include/Acknowledgements.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Pyth

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.06116-b31b1b.svg)](https://arxiv.org/abs/2602.06116) | **The Host Galaxies of Active Galactic Nuclei with Direct Black Hole Mass Measurements**  |
|| V. N. Bennert, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2026-02-09*|
|*Comments*| *31 pages, 12 figures (figures 5-9 to appear as figure set in the online version), 5 tables, accepted by the Astrophysical Journal*|
|**Abstract**|            Reverberation mapping (RM) determines the mass of black holes (BH) in active galactic nuclei (AGNs) by resolving the BH gravitational sphere of influence in the time domain. Recent RM campaigns yielded direct BH masses through dynamical modeling for a sample of 32 objects, spanning a wide range of AGN luminosities and BH masses. In addition, accurate BH masses have been determined by spatially resolving the broad-line region with GRAVITY for a handful of AGNs. Here, we present a detailed analysis of Hubble Space Telescope images using surface-brightness profile fitting with state-of-the-art programs. We derive AGN luminosity and host-galaxy properties, such as radii and luminosities for spheroid, disk, and bar (if present). The spheroid effective radii were used to measure stellar velocity dispersion from integral-field spectroscopy. Since the BH masses of our sample do not depend on any assumption of the virial factor needed in single-epoch spectroscopic mass estimates, we can show that the resulting scaling relations between the mass of the supermassive BHs and their host galaxies match those of quiescent galaxies, naturally extending to lower masses in these (predominantly) spiral galaxies. We find that the inner AGN orientation, as traced by the broad-line region inclination angle, is uncorrelated with the host-galaxy disk. Our sample has the most direct and accurate MBH measurements of any AGN sample and provides a fundamental local benchmark for studies of the evolution of massive black holes and their host galaxies across cosmic time.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.06551-b31b1b.svg)](https://arxiv.org/abs/2602.06551) | **AB Aur, a Rosetta stone for studies of planet formation (IV): C/O estimates from CS and SO interferometric observations**  |
|| P. Rivière-Marichalar, et al. -- incl., <mark>D. Semenov</mark> |
|*Appeared on*| *2026-02-09*|
|*Comments*| *11 pages, 13 figures*|
|**Abstract**|            Context. Protoplanetary disks are the birthplace of planets. As such, they set the initial chemical abundances available for planetary atmosphere formation. Thus, studying elemental abundances, molecular compositions, and abundance ratios in protoplanetary disks is key to linking planetary atmospheres to their formation sites. Aims. We aim to derive the sulfur abundance and the C/O ratio in the AB Aur disk using interferometric observations of CS and SO. Methods. New NOEMA observations of CS 3-2 towards AB Aur are presented. We used velocity-integrated intensity maps to determine the inclination and position angles. Keplerian masks were constructed for all observed species to assess the presence of non-Keplerian motions. We use the CS/SO ratio to study the C/O ratio. We compare our present and previous interferometric observations of AB Aur with a NAUTILUS disk model to gain insight into the S elemental abundance and C/O ratio. Results. We derive an observational CS/SO ratio ranging from 1.8 to 2.6. Only NAUTILUS models with C/O > 1 can reproduce such ratios. The comparison with models points to strong sulfur depletion, with [S/H]=8e-8, but we note that no single model can simultaneously fit all observed species.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.06463-b31b1b.svg)](https://arxiv.org/abs/2602.06463) | **ELUCID-DESI I: A Parallel MPI Implementation of the Initial Condition Solver for Large-Scale Reconstruction Simulations**  |
|| W. Hong, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2026-02-09*|
|*Comments*| **|
|**Abstract**|            We present a highly scalable, MPI-parallelized framework for reconstructing the initial cosmic density field, designed to meet the computational demands of next-generation cosmological simulations, particularly the upcoming ELUCID-DESI simulation based on DESI BGS data. Building upon the Hamiltonian Monte Carlo approach and the FastPM solver, our code employs domain decomposition to efficiently distribute memory between nodes. Although communication overhead increases the per-step runtime of the MPI version by roughly a factor of eight relative to the shared-memory implementation, our scaling tests-spanning different particle numbers, core counts, and node layouts-show nearly linear scaling with respect to both the number of particles and the number of CPU cores. Furthermore, to significantly reduce computational costs during the initial burn-in phase, we introduce a novel ``guess'' module that rapidly generates a high-quality initial density field. The results of the simulation test confirm substantial efficiency gains: for $256^3$ particles, 53 steps ($\sim$54 CPU hours) are saved; for $1024^3$, 106 steps ($\sim$7500 CPU hours). The relative gain grows with the number of particles, rendering large-volume reconstructions computationally practical for upcoming surveys, including our planned ELUCID-DESI reconstruction simulation with $8192^3$ particles, with a rough estimation of 720 steps ($\sim$37,000,000 CPU hours).         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.06878-b31b1b.svg)](https://arxiv.org/abs/2602.06878) | **Antenna for the detection of electromagnetic audio-band disturbances on-board LISA**  |
|| <mark>D. Serrano</mark>, et al. |
|*Appeared on*| *2026-02-09*|
|*Comments*| *27 pages, 10 figures, 1 table*|
|**Abstract**|            The LISA mission will be the first observatory to detect gravitational waves from space within the millihertz frequency band. Magnetic forces have an important impact on the instrument's sensitivity below the millihertz. Hence, monitoring the magnetic environment within each of the LISA spacecrafts is of utmost importance. In this Letter we present the characterization of the coils that were used in LISA Pathfinder (LPF) when operating as magnetic sensors in the audio frequency band. The necessity of implementing this type of magnetometer is presented in order to monitor high frequency magnetic signals from the electronics on-board. We show that the LPF coils have a performance one order of magnitude better than the current requirements set by the LISA mission at the low end of the audio-band frequency. The LPF coils are able to measure a magnetic noise level of 1.45 $\rm pT/\sqrt{\rm{Hz}}$ at 50 Hz and 0.17 $\rm pT/\sqrt{\rm{Hz}}$ at 500 Hz. Additionally, the LPF coils can reach a magnetic noise floor of 0.1 $\rm pT/\sqrt{\rm{Hz}}$ at frequencies above 1 kHz.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2602.06116/./l335000_600_sersicdisk_1.png', 'tmp_2602.06116/./l1501_375_sersicdisk_1.png', 'tmp_2602.06116/./l535_675_sersicdiskbar_1.png', 'tmp_2602.06116/./l590_570_sersicdisk_1.png', 'tmp_2602.06116/./l10448888_1000_sersicdiskbar_1.png', 'tmp_2602.06116/./BLR_inclination.png', 'tmp_2602.06116/./BLR_opening.png', 'tmp_2602.06116/./MBH_LI_sph.png', 'tmp_2602.06116/./MBH_LV_sph.png']
copying  tmp_2602.06116/./l335000_600_sersicdisk_1.png to _build/html/
copying  tmp_2602.06116/./l1501_375_sersicdisk_1.png to _build/html/
copying  tmp_2602.06116/./l535_675_sersicdiskbar_1.png to _build/html/
copying  tmp_2602.06116/./l590_570_sersicdisk_1.png to _build/html/
copying  tmp_2602.06116/./l10448888_1000_sersicdiskbar_1.png to _build/html/
copying  tmp_2602.06116/./BLR_inclination.png to _build/html/
copying  tmp_2602.06116/./BLR_opening.png to _build/html/
copying  tmp_2602.06116/./MBH_LI_sph.png to _build/html/
copying  tmp_2602.06116/./MBH_LV_sph.png to _build/html/
exp

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\mbh}{M_{\rm BH}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\etal}{\textit{et al.}}$
$\newcommand{\Hz}{{\rm Hz}}$
$\newcommand{\erg}{{\rm erg}}$
$\newcommand{\yr}{{\rm yr}}$
$\newcommand{\cm}{{\rm cm}}$
$\newcommand{\s}{{\rm s}}$
$\newcommand{\Mpc}{{\rm Mpc}}$
$\newcommand{\ls}{L_{\rm sph}}$
$\newcommand{\lh}{L_{\rm host}}$
$\newcommand{\ms}{M_{\rm sph}}$
$\newcommand{\mss}{M_{\rm sph, \star}}$
$\newcommand{\mts}{M_{\rm host, \star}}$
$\newcommand{\kms}{{\rm km s}^{-1}}$
$\newcommand{\perMpc}{{\rm Mpc}^{-1}}$
$\newcommand{\hh}{H_{2}}$
$\newcommand{\um}{\mum~}$
$\newcommand{\feii}{[Fe \textsc{ii}]}$
$\newcommand{\pab}{Pa\beta}$
$\newcommand{\molhy}{H_2}$
$\newcommand{\brg}{Br\gamma}$
$\newcommand{\ha}{H\alpha}$
$\newcommand{\hb}{H\beta}$
$\newcommand{\hg}{H\gamma}$
$\newcommand{\oiii}{[O \textsc{iii}]}$
$\newcommand{\oii}{[O \textsc{ii}]}$
$\newcommand{\oi}{[O \textsc{i}]}$
$\newcommand{\nii}{[N \textsc{ii}]}$
$\newcommand{\neiii}{[Ne \textsc{iii}]}$
$\newcommand{\sii}{[S \textsc{ii}]}$
$\newcommand{\mdyn}{M_{\rm sph, dyn}}$
$\newcommand{\mstar}{M_{\star}}$
$\newcommand{\msun}{M_{\odot}}$
$\newcommand{\lsun}{L_{\odot}}$
$\newcommand{\lya}{Ly\alpha}$</div>



<div id="title">

# The Host Galaxies of Active Galactic Nuclei with Direct Black Hole Mass Measurements$\footnote{Based on observations made with the NASA/ESA Hubble Space Telescope,obtained at the Space Telescope Science Institute, which is operated by theAssociation of Universities for Research in Astronomy, Inc., under NASAcontract NAS5-26555.These observations are associated with programs\# 17103 and 17063.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.06116-b31b1b.svg)](https://arxiv.org/abs/2602.06116)<mark>Appeared on: 2026-02-09</mark> -  _31 pages, 12 figures (figures 5-9 to appear as figure set in the online version), 5 tables, accepted by the Astrophysical Journal_

</div>
<div id="authors">

Vardha~N.~Bennert, et al.

</div>
<div id="abstract">

**Abstract:** Reverberation mapping (RM) determines the mass of black holes (BH) in active galactic nuclei (AGNs) byresolving the BH gravitational sphere of influence in the time domain. RecentRM campaigns yielded direct BH masses through dynamical modeling for a sample of 32 objects, spanning a widerange of AGN luminosities and BH masses. In addition,accurate BH masses have been determined by spatially resolving the broad-line region with GRAVITY for a handful of AGNs.Here, we present a detailed analysis of $_ Hubble Space Telescope_$ images using surface-brightness profile fitting with state-of-the-art programs.We derive AGN luminosity and host-galaxy properties, such as radii andluminosities for spheroid, disk, and bar (if present).The spheroid effective radiiwere used to measure stellar velocity dispersion from integral-field spectroscopy.Since the BH masses of our sample do not depend on any assumption of thevirial factor needed in single-epoch spectroscopic mass estimates, we can show that the resultingscaling relations between the mass of the supermassive BHs and their hostgalaxies match those of quiescent galaxies, naturally extending to lower massesin these (predominantly) spiral  galaxies. We find that the inner AGN orientation, as traced by the broad-line region inclination angle, is uncorrelatedwith the host-galaxy disk.Our sample has the most direct and accurate $\mbh$ measurementsof any AGN sample and provides a fundamental local benchmark for studies of theevolution of massive black holes and their host galaxies across cosmic time.

</div>

<div id="div_fig1">

<img src="tmp_2602.06116/./l335000_600_sersicdisk_1.png" alt="Fig4.1" width="20%"/><img src="tmp_2602.06116/./l1501_375_sersicdisk_1.png" alt="Fig4.2" width="20%"/><img src="tmp_2602.06116/./l535_675_sersicdiskbar_1.png" alt="Fig4.3" width="20%"/><img src="tmp_2602.06116/./l590_570_sersicdisk_1.png" alt="Fig4.4" width="20%"/><img src="tmp_2602.06116/./l10448888_1000_sersicdiskbar_1.png" alt="Fig4.5" width="20%"/>

**Figure 4. -** *Surface-Photometry Fits.*
From left to right: observed HST image ("data", including
scale bar of 5 arcseconds and North-East directions); best-fit \texttt{galight}/\texttt{lenstronomy} model ("model"); PSF-subtracted image ("data—point source"); residual image after subtraction of best-fit model
from data, divided by the noise level ("normalized residual");
and surface-brightness profile
(data = black circles, model = red line, PSF = blue line,
spheroid = yellow line; if present: disk = purple line, bar = orange line).
The surface-brightness profile is shown for illustration only, as the fits were performed on the 2D image.
Surface-brightness values are given in the plane of the sky (total light within circular aperture; the x-axis is based on a circularized radius).
Note that all images
are displayed as observed with HST (see Table \ref{tab:hst}) and fitted by \texttt{lenstronomy}.
Each row of images corresponds to one object (as labeled on the y-axis of
the leftmost panel).
  (*fig:lenstronomy1*)

</div>
<div id="div_fig2">

<img src="tmp_2602.06116/./BLR_inclination.png" alt="Fig2.1" width="50%"/><img src="tmp_2602.06116/./BLR_opening.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** _ Left:_*BLR Inclination vs. Host Inclination.*
        Comparison between inclination of the BLR, based on
        \texttt{CARAMEL} and GRAVITY modeling, and host-galaxy disk inclination, for our sample of 34 RM AGNs (classical bulges in yellow, pseudo-bulges in blue).
        There is no correlation between the inclination
        of the central AGN and the large-scale host-galaxy disk;
        if at all, there is an indication of a weak anti-correlation
(Pearson correlation coefficient of $-0.16$).
        _ Right:_*BLR Inclination vs. BLR Opening Angle.*
        Similar to the left panel, but now comparing
 inclination and opening angle of the BLR, based on
        \texttt{CARAMEL} and GRAVITY modeling.
Both parameters are correlated (Pearson correlation coefficient of
0.71). The fitted relation is shown as a shaded gray stripe
corresponding to the  68\%(1-sigma) confidence
interval of the linear regression.
The correlation is
driven by the absence of double-peaked lines in the sample.
In other words, in modeling the data, the opening angle of the BLR cannot be much smaller than the
inclination angle, while still reproducing the observed single-peaked lines. (*fig:inclination*)

</div>
<div id="div_fig3">

<img src="tmp_2602.06116/./MBH_LI_sph.png" alt="Fig1.1" width="50%"/><img src="tmp_2602.06116/./MBH_LV_sph.png" alt="Fig1.2" width="50%"/>

**Figure 1. -** *$\mbh$--Spheroid Luminosity Relation.*
In the left panel, the relation between $\mbh$ and spheroid luminosity
in the $I$-band is shown, in the right panel, the same but in the $V$-band.
Grey data points are quiescent galaxies
\citep[ellipticals or spirals/S0 with classical bulges;][]{Kormendy_Ho:2013};
colored data points are our sample (classical bulges in yellow, pseudo-bulges in blue).
The fitted relation is shown as a shaded gray stripe
corresponding to the  68\%(1-sigma) confidence
interval of the linear regression.
Since our sample consists of local AGNs with directly measured $\mbh$ through dynamical modeling which is free of assumptions
of the virial factor, we can show that
AGNs follow the same scaling relations as those
of quiescent galaxies. (*fig:relation*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.06116"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# AB Aur, a Rosetta stone for studies of planet formation (IV): C/O estimates from CS and SO interferometric observations.

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.06551-b31b1b.svg)](https://arxiv.org/abs/2602.06551)<mark>Appeared on: 2026-02-09</mark> -  _11 pages, 13 figures_

</div>
<div id="authors">

P. Rivière-Marichalar, et al. -- incl., <mark>D. Semenov</mark>

</div>
<div id="abstract">

**Abstract:** Protoplanetary disks are the birthplace of planets. As such, they set the initial chemical abundances available for planetary atmosphere formation. Thus, studying elemental abundances, molecular compositions, and abundance ratios in protoplanetary disks is key to linking planetary atmospheres to their formation sites. We aim to derive the sulfur abundance and the C/O ratio in the AB Aur disk using interferometric observations of CS and SO. New NOEMA observations of CS 3-2 towards AB Aur are presented. We used velocity-integrated intensity maps to determine the inclination and position angles. Keplerian masks were constructed for all observed species to assess the presence of non-Keplerian motions. We use the CS/SO ratio to study the C/O ratio. We compare our present and previous interferometric observations of AB Aur with a NAUTILUS disk model to gain insight into the S elemental abundance and C/O ratio. We derive an observational CS/SO ratio ranging from 1.8 to 2.6. Only NAUTILUS models with C/O $\gtrsim$ 1 can reproduce such ratios. The comparison with models points to strong sulfur depletion, with [ S/H ] =8 $\times$ 10 $\rm ^{-8}$ , but we note that no single model can simultaneously fit all observed species.

</div>

<div id="div_fig1">

<img src="tmp_2602.06551/./ABAur_CS_moments_gridplot.png" alt="Fig10" width="100%"/>

**Figure 10. -** From left to right: zeroth-, first, and second-moment map of CS 3-2 observed with NOEMA. The first- and second-moment maps were computed using a 3$\sigma$ threshold to avoid noisy channels. Contours in the zeroth- and second-moment maps are shown for 0.25, 0.5, 0.75, and 0.9 of the integrated intensity map peak. The white dashed ellipse in the zeroth-moment map shows the area used to compute the integrated spectrum shown in Fig. \ref{Fig:CS_spectrum}. (*Fig:CS_moments*)

</div>
<div id="div_fig2">

<img src="tmp_2602.06551/./ABAur_CS_spec.png" alt="Fig1" width="100%"/>

**Figure 1. -** CS spectrum extracted from a 2$\arcsec \times $1.8$\arcsec$ ellipse with PA=237 $^\circ$ centered on the CS emission disk (see Fig. \ref{Fig:CS_moments}, left panel). The blue curve shows a 2-Gaussian fit to the data, with the black dashed curves representing the individual Gaussians. (*Fig:CS_spectrum*)

</div>
<div id="div_fig3">

<img src="tmp_2602.06551/./radprof.png" alt="Fig3" width="100%"/>

**Figure 3. -** Azimuthally averaged radial profile of the CS 3-2 velocity integrated intensity map (black line) and the continuum at 2 mm (blue line). The shaded areas account for the uncertainties in the profiles. (*Fig:CS_radprof*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.06551"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

122  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

16  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
